In [5]:
# ---------- Импорты ----------
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
import time
import warnings
import json
import sys

# ---------- Путь к модели ----------
MODEL_PATH = '/home/skoltsov/qwen_1_5b/qwen_model/'

# ---------- Параметры инференса ----------
MAX_NEW_TOKENS = 3000
test_percentage = 1
BATCH_SIZE = 62

# ---------- Игнорирование предупреждений ----------
warnings.filterwarnings("ignore", message="Current model requires 128 bytes of buffer for offloaded layers*")

# ---------- Оптимизация скорости ----------
torch.backends.cudnn.benchmark = True

# ---------- Показ GPU памяти ----------
def print_gpu_memory():
    if torch.cuda.is_available():
        print(f"🧠 Использовано GPU памяти: {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
        print(f"🛡 Зарезервировано GPU памяти: {torch.cuda.memory_reserved() / (1024**2):.2f} MB")
    else:
        print("CUDA недоступна.")

# ---------- Загрузка токенизатора и модели ----------
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    padding_side="left"
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="cuda"
)

if hasattr(model.config, "use_sliding_window_attention"):
    model.config.use_sliding_window_attention = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

# ---------- Попытка компиляции модели ----------
try:
    model = torch.compile(model)
except Exception as e:
    print(f"torch.compile не сработал: {e}")

# ---------- Загрузка данных ----------
test_data = pd.read_excel("/home/skoltsov/qwen_1_5b/data_balansed/test_x_balans.xlsx")
test_data = test_data.sample(frac=test_percentage, random_state=42).reset_index(drop=True)
print('📄 Количество уравнений:', len(test_data))

# ---------- Функция генерации промптов ----------
def build_prompts(equations):
    prompts = []
    for eq in equations:
        prompt = (
            f'Solve the differential equation: {eq}. '
            f'It is must to be provided the final decision in LaTeX format, enclosed in \\boxed{{}}'
        )
        prompts.append(prompt)
    return prompts

# ---------- Функция решения уравнений батчем ----------
def solve_equations_batch(equations_batch, max_new_tokens=MAX_NEW_TOKENS):
    prompts = build_prompts(equations_batch)
    
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    start_time = time.time()

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=True
        )

    end_time = time.time()
    inference_time = end_time - start_time

    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    return generated_texts, inference_time

# ---------- Функция отображения прогрессбара ----------
def print_progress(current, total, elapsed_time):
    progress = (current / total)
    bar_length = 30
    block = int(bar_length * progress)
    time_per_batch = elapsed_time / current if current else 0
    eta = time_per_batch * (total - current)

    bar = "█" * block + "-" * (bar_length - block)
    percent = progress * 100
    sys.stdout.write(f"\r📈 Прогресс: |{bar}| {percent:.2f}% ({current}/{total}) | ⏳ Прошло: {elapsed_time:.1f}c | ⏱ ETA: {eta:.1f}c")
    sys.stdout.flush()



t1= time.time()

# ---------- Основной цикл обработки + сбор результатов ----------
output_filename = "Qwen_1_5b_inference_results_3000tok_balansed.json"
results = []

num_batches = (len(test_data) + BATCH_SIZE - 1) // BATCH_SIZE
print(f"🔢 Общее количество батчей: {num_batches}")

start_global_time = time.time()

for batch_idx, start_idx in enumerate(range(0, len(test_data), BATCH_SIZE)):
    end_idx = start_idx + BATCH_SIZE
    batch = test_data.iloc[start_idx:end_idx]
    equations_batch = batch["equation"].tolist()

    solutions, inference_time = solve_equations_batch(equations_batch)

    for i, (eq, sol) in enumerate(zip(equations_batch, solutions)):
        #print("\n\nУравнение:")
        #print(eq)
        #print("\nОтвет модели:")
        #print(sol)
        #print("-" * 60)

        results.append({
            "equation": eq,
            "true_answer": batch.iloc[i].get('true_answer', ''),
            "generated_answer": sol,
            "type_eq": batch.iloc[i].get('type_eq', ''),  # <-- Новая строка
        })

    print(f"\n⏱️ Время инференса батча: {inference_time:.2f} секунд")
    print_gpu_memory()
    print("=" * 100)

    # Сохраняем после каждого батча
    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    elapsed_global_time = time.time() - start_global_time
    print_progress(batch_idx + 1, num_batches, elapsed_global_time)

print("\n\n✅ Все результаты инференса сохранены в", output_filename)
t2= time.time()

print('Время работы модели: ', t2-t1)

torch.compile не сработал: Dynamo is not supported on Python 3.12+
📄 Количество уравнений: 1710
🔢 Общее количество батчей: 28

⏱️ Время инференса батча: 196.06 секунд
🧠 Использовано GPU памяти: 2953.28 MB
🛡 Зарезервировано GPU памяти: 11962.00 MB
📈 Прогресс: |█-----------------------------| 3.57% (1/28) | ⏳ Прошло: 197.0c | ⏱ ETA: 5320.2c
⏱️ Время инференса батча: 195.17 секунд
🧠 Использовано GPU памяти: 2953.28 MB
🛡 Зарезервировано GPU памяти: 11962.00 MB
📈 Прогресс: |██----------------------------| 7.14% (2/28) | ⏳ Прошло: 392.7c | ⏱ ETA: 5104.9c
⏱️ Время инференса батча: 197.72 секунд
🧠 Использовано GPU памяти: 2953.28 MB
🛡 Зарезервировано GPU памяти: 12534.00 MB
📈 Прогресс: |███---------------------------| 10.71% (3/28) | ⏳ Прошло: 590.7c | ⏱ ETA: 4922.3c
⏱️ Время инференса батча: 192.71 секунд
🧠 Использовано GPU памяти: 2953.28 MB
🛡 Зарезервировано GPU памяти: 12534.00 MB
📈 Прогресс: |████--------------------------| 14.29% (4/28) | ⏳ Прошло: 783.5c | ⏱ ETA: 4700.9c
⏱️ Время инфере